In [1]:
%load_ext autoreload
%autoreload 2

#%pylab notebook
%matplotlib inline
from pylab import *
from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold')
rcParams["figure.facecolor"] = 'white'
from scipy.optimize import curve_fit

In [2]:
rcParams['figure.dpi'] = 150
rcParams['figure.facecolor'] = 'white'

In [3]:
import sys, numpy as np, scipy as sc, warnings, os#, healpy as H
sys.path.append('/Users/sraghunathan/Research/SPTPol/analysis/git/DRAFT/modules/')
import foregrounds as fg, misc, flatsky, misc, exp_specs

#import matplotlib.cbook
warnings.filterwarnings('ignore',category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=matplotlib.cbook.mplDeprecation)
#warnings.filterwarnings('ignore', category=OptimizeWarning)

In [4]:
#params
expname = 's4wide'
#expname = 'cmbhd'
#expname = 's4deep'
#expname = 's4deepv3r025' #20201019
specs_dic, corr_noise_bands, rho, corr_noise = exp_specs.get_exp_specs(expname)
#freqarr = sorted( specs_dic.keys() )
freqarr = [93, 145, 278]
print(freqarr)
nc = len( freqarr )
TParr = ['T', 'P']

#for Neff: T/P available
param_dict = {}
nside, lmax = 2048, 5000
#which_sim_set = '202002_foregrounds_extragalactic_cmb_tophat'
which_sim_set = '202102_design_tool_input'

param_dict['lmax'] = lmax
param_dict['Dlfile_len'] = '/Users/sraghunathan/Research/SPTPol/analysis/git/DRAFT/data/output_planck_r_0.0_2015_cosmo_lensedCls.dat'

[93, 145, 278]


In [5]:
#beam and noise arr
beamarr = []
noisearr_T, elkneearr_T, alphakneearr_T = [], [], []
noisearr_P, elkneearr_P, alphakneearr_P = [], [], []
for freq in freqarr:
    beam_arcmins, white_noise_T, elknee_T, alphaknee_T, white_noise_P, elknee_P, alphaknee_P = specs_dic[freq]
    beamarr.append(beam_arcmins)
    noisearr_T.append(white_noise_T)
    noisearr_P.append(white_noise_P)
    elkneearr_T.append(elknee_T)
    elkneearr_P.append(elknee_P)
    alphakneearr_T.append(alphaknee_T)
    alphakneearr_P.append(alphaknee_P)    

print(elkneearr_T)

#collect beam and noise into a dic; elknee and alpha into a dic
beam_noise_dic = {}
elknee_dic = {}
for TP in TParr:
    beam_noise_dic[TP] = {}
    elknee_dic[TP] = {} 
    if TP == 'T':
        freqarr, beamarr, noisearr, elkneearr, alphakneearr = freqarr, beamarr, noisearr_T, elkneearr_T, alphakneearr_T
    elif TP == 'P':
        freqarr, beamarr, noisearr, elkneearr, alphakneearr = freqarr, beamarr, noisearr_P, elkneearr_P, alphakneearr_P

    for (freq, beam, noise, elknee, alphaknee) in zip(freqarr, beamarr, noisearr, elkneearr, alphakneearr):
        beam_noise_dic[TP][freq] = [beam, noise]
        elknee_dic[TP][freq] = [elknee, alphaknee]

#get beams
bl_dic = misc.get_beam_dic(freqarr, beam_noise_dic['T'], param_dict['lmax']+1)
print(bl_dic.keys())
if (0):
    for freq in freqarr:
        plot(bl_dic[freq], label = freq)
    legend(loc = 1)

[1932.0, 3917.0, 6792.0]
dict_keys([93, 145, 278])


In [6]:
ell_norm = 80
def power_law(ell, A, alpha, ell_norm = 80.):
    #print(A, alpha)#, end = ' ')
    fit = A * ((ell / ell_norm) ** alpha)
    badinds = np.where((fit == np.inf) | (fit == np.nan))[0]
    fit[badinds]=0.
    return fit
def perform_fit(el, cl, ell_norm = 80, which_spec = 'TT'):
    dl_fac = (el * (el + 1))/2/np.pi
    dl = cl * dl_fac
    badinds = np.where((dl == np.inf) | (dl == np.nan))[0]
    dl[badinds]=0.
    amp_ini = dl[el == ell_norm][0]
    if which_spec == 'EE':
        alpha_ini = -.4
    elif which_spec == 'TT':
        alpha_ini = -.3
    #print(amp_ini*0.1, amp_ini*3., alpha_ini-0.2, alpha_ini+0.2); sys.exit()
    if which_spec == 'TT':
        amp_low_fac, amp_high_fac = 0.95, 1.05 #0.1, 3.
        delta_alpha = 0.1
    else:
        amp_low_fac, amp_high_fac = 0.95, 1.05 #0.1, 3.
        delta_alpha = 0.2
    pars, cov = curve_fit(f=power_law, xdata=el, ydata=dl, p0=[amp_ini, alpha_ini], bounds = ((amp_ini*amp_low_fac, alpha_ini-delta_alpha), (amp_ini*amp_high_fac, alpha_ini+delta_alpha)))
    dl_fit = power_law(el, pars[0], pars[1])
    cl_fit = dl_fit / dl_fac

    return cl_fit, pars

In [13]:
#get and store dust and sync values for both S4-Wide and S4-Ultra deep fields
cl_gal_dic = {}
which_spec = 'EE' #'TT' #'EE'# 'TT' #'TT'
max_l = 1000
exparr = ['s4wide', 's4deep']
for expiter, expname in enumerate( exparr ):
    if expiter == 0: #S4-wide
        data_folder = '/Volumes/data_PHD_WD_babbloo/s4/cmbs4/map_based_simulations/%s/4096/' %(which_sim_set)
        param_dict['cl_gal_dic_dust_fname'] = '%s/dust/0000/s4like_mask_v2/cls_galactic_sims_dust_nside%s_lmax%s_cos_el_40.npy' %(data_folder, nside, lmax)
        param_dict['cl_gal_dic_sync_fname'] = '%s/synchrotron/0000/s4like_mask_v2/cls_galactic_sims_sync_nside%s_lmax%s_cos_el_40.npy' %(data_folder, nside, lmax)
        extra_str = 'pySM3/s4like_mask_v2/'
        which_gal_mask = 2
    elif expiter == 1: #S4-delensing
        data_folder = '/Volumes/data_PHD_WD_babbloo/s4/cmbs4/map_based_simulations/%s/4096/' %(which_sim_set)
        param_dict['cl_gal_dic_dust_fname'] = '%s/dust/0000/s4delensing_mask/cls_galactic_sims_dust_nside%s_lmax%s_delensing.npy' %(data_folder, nside, lmax)
        param_dict['cl_gal_dic_sync_fname'] = '%s/synchrotron/0000/s4delensing_mask/cls_galactic_sims_sync_nside%s_lmax%s_delensing.npy' %(data_folder, nside, lmax)
        extra_str = 'pySM3/s4delensing_mask/'
        which_gal_mask = 0
    cl_gal_dic[expname] = {}
    for freq in freqarr:
        el, cl_dust = fg.get_cl_galactic(param_dict, 'dust', freq, freq, which_spec = which_spec, which_gal_mask = which_gal_mask, bl_dic = bl_dic, use_sed_scaling = True, use_power_law_fit = False)
        el, cl_sync = fg.get_cl_galactic(param_dict, 'sync', freq, freq, which_spec = which_spec, which_gal_mask = which_gal_mask, bl_dic = bl_dic)
        if (1):
            el = el[:max_l]
            cl_sync = cl_sync[:max_l]
            cl_dust = cl_dust[:max_l]
        cl_dust_fit, dust_pars = perform_fit(el, cl_dust, ell_norm = ell_norm)
        cl_sync_fit, sync_pars = perform_fit(el, cl_sync, ell_norm = ell_norm)
        cl_gal_dic[expname][freq] = {}
        cl_gal_dic[expname][freq]['dust'] = [el, cl_dust, cl_dust_fit, dust_pars]
        cl_gal_dic[expname][freq]['sync'] = [el, cl_sync, cl_sync_fit, sync_pars]
print(cl_gal_dic.keys())

dict_keys(['s4wide', 's4deep'])


In [14]:
#CAMB output for plotting
camb_file = param_dict['Dlfile_len']
Tcmb = 2.725
el_camb = np.loadtxt(camb_file, usecols = [0])
dl_camb = np.loadtxt(camb_file, usecols = [1,2,3,4])

cl_camb = ( Tcmb**2. * dl_camb * 2 * np.pi ) / ( el_camb[:,None] * (el_camb[:,None] + 1) )
cl_camb *= 1e12
cl_TT, cl_EE, cl_BB, cl_TE = cl_camb.T

In [ ]:
#plot
clf()
fig = figure(figsize=(10., 6.))
subplots_adjust(wspace=0.05)
xmin, xmax = 10, 1000
ymin, ymax = 1e-8, 10.
exp_dic = {'s4wide': ['S4-Wide', 'red'], 's4deep': ['S4-Ultra deep', 'navy']}
dust_sync_freq_dic = {'dust':145, 'sync':93}
sync_freq = 93
for sbpl, dust_or_sync in enumerate(['dust', 'sync']):
    print('\t%s' %(dust_or_sync.capitalize()))
    ax=subplot(2,2,sbpl+1, yscale = 'log')
    plot(el_camb, cl_TT, 'gray', lw = 2., alpha = 0.3, zorder = -1000.)#, label = r'%s' %(which_spec))
    for expcntr, expname in enumerate( exparr ):
        whichfreq = dust_sync_freq_dic[dust_or_sync]
        explabval, colorval = exp_dic[expname]
        el, cl, cl_fit, pars = cl_gal_dic[expname][whichfreq][dust_or_sync]
        print('\t\t%s (Band = %s GHz): Amplitude = %.3f, Slope = %.3f' %(explabval, whichfreq, pars[0], pars[1]))
        plot(el, cl, color = colorval, ls = ':', alpha = 0.5)
        plot(el, cl_fit, color = colorval, ls = '-', label = explabval)
    if sbpl == 0:
        plot([], [], 'k:', alpha = 0.5, label = r'pySM')
        fitlabstr = r'Fit: A$\left(\ell/\ell_{\rm norm}\right)^{\alpha}$'
        fitlabstr = r'Fit: A$\left(\ell/%s\right)^{\alpha}$' %(ell_norm)
        plot([], [], 'k-', label = fitlabstr)
        ylabel(r'$C_{\ell}\ [\mu {\rm K}^{2}]$', fontsize = 14)
        legend(loc = 4, ncol = 2, fontsize = 10)
    else:
        setp(ax.get_yticklabels(which = 'both'), visible=False)
    title(r'%s (Band = %s GHz): %s' %(dust_or_sync.capitalize(), whichfreq, which_spec))
    xlim(xmin, xmax); ylim(ymin, ymax)
    if (1):
        #xlabel(r'Multipole $\ell$', fontsize = 14)
        setp(ax.get_xticklabels(which = 'both'), visible=False)
    axvline(ell_norm, ls = '-.')
    grid(True, which='major', axis = 'x', lw = 0.5, alpha = 0.1)
    #grid(True, which='minor', axis = 'x', lw = 0.25, alpha = 0.1)
    grid(True, which='both', axis = 'y', lw = 0.5, alpha = 0.1)
    grid(True, which='minor', axis = 'y', lw = 0.25, alpha = 0.1)

    ax=subplot(2,2,sbpl+3, yscale = 'log')
    el_s4wide, cl_s4wide, cl_fit_s4wide, pars_s4wide = cl_gal_dic['s4wide'][whichfreq][dust_or_sync]
    el_s4deep, cl_s4deep, cl_fit_s4deep, pars_s4deep = cl_gal_dic['s4deep'][whichfreq][dust_or_sync]
    plot(el_s4wide, cl_s4wide/cl_s4deep, 'k', ls = ':', alpha = 0.5)
    plot(el_s4wide, cl_fit_s4wide/cl_fit_s4deep, 'k-')
    xlim(xmin, xmax); ylim(0.1, 100.)
    xlabel(r'Multipole $\ell$', fontsize = 14)
    axvline(ell_norm, ls = '-.');
    grid(True, which='major', axis = 'x', lw = 0.5, alpha = 0.1)
    #grid(True, which='minor', axis = 'x', lw = 0.25, alpha = 0.1)
    grid(True, which='both', axis = 'y', lw = 0.5, alpha = 0.1)
    grid(True, which='minor', axis = 'y', lw = 0.25, alpha = 0.1)
    if sbpl == 0:
        ylabel(r'S4-Wide/S4-Ultra deep', fontsize = 12)
    else:
        setp(ax.get_yticklabels(which = 'both'), visible=False)
    
show(); sys.exit()
    

	Dust
		S4-Wide (Band = 145 GHz): Amplitude = 1.231, Slope = -0.371
		S4-Ultra deep (Band = 145 GHz): Amplitude = 0.457, Slope = -0.328
	Sync
		S4-Wide (Band = 93 GHz): Amplitude = 0.010, Slope = -0.400
		S4-Ultra deep (Band = 93 GHz): Amplitude = 0.002, Slope = -0.400


<Figure size 900x600 with 0 Axes>

In [ ]:
mask_str_dic = None
#alpha_dic = {93: 0.3, 145: 0.6, 225: 0.8, 278: 1.}
alpha_dic = {93: .5, 145: 1., 225: 1., 278: 1.}
lw_dic = {93: 1., 145: 1., 225: 1.5, 278: 2.}
ls_dic = {93: '-.', 145: '-', 225: '--', 278: ':'}

tot_mask_iter = 4
color_dic = {0:'navy', 1: 'green', 2: 'goldenrod', 3: 'darkred'}

tot_mask_iter = 5
color_dic = {0:'navy', 1: 'green', 2: 'goldenrod', 3: 'orangered', 4: 'darkred'}

tot_mask_iter = 4
color_dic = {0:'navy', 1: 'green', 2: 'goldenrod', 3: 'darkred'}
reqd_masks = []
if extra_str.find('s4like_mask_v2')>-1 or extra_str.find('s4delensing')>-1:
    tot_mask_iter = 6
    if extra_str.find('s4delensing')>-1:
        reqd_masks = [0]
    else:
        #reqd_masks = [0, 1, 2, 3, 4, 5]
        reqd_masks = [2]#, 5]
    which_spec = 'TT' #EE'#'TT' #EE' #TT'

In [ ]:
show_sync = 1
clf()
if show_sync:
    fig = figure(figsize=(8., 4.))
    subplots_adjust(wspace=0.05)
#xscale_val = 'log' ##None#'log' #None
xscale_val = None
freqarr=[93, 145]
tot_freq_iter = len(freqarr)
which_gal_mask = 2 
plot_done = []
xmin = 10
xmax = 1000#7100 #lmax+100
ymin, ymax = 1e-10, 10.
Tdust, beta_dust = 20., 1.54 #https://pysm3.readthedocs.io/en/latest/models.html#dust
#Tdust, beta_dust = 20., 1.3 #https://pysm3.readthedocs.io/en/latest/models.html#dust
color_dic = {}
color_dic[(93, 93)] = 'navy'
color_dic[(145, 145)] = 'red'
for freq1 in freqarr:
    for freq2 in freqarr:

        if (freq1, freq2) in plot_done: continue

        if freq1 != freq2: continue

        el, cl_dust = fg.get_cl_galactic(param_dict, 'dust', freq1, freq2, which_spec = which_spec, which_gal_mask = which_gal_mask, bl_dic = bl_dic, use_sed_scaling = True, use_power_law_fit = False)
        el, cl_sync = fg.get_cl_galactic(param_dict, 'sync', freq1, freq2, which_spec = which_spec, which_gal_mask = which_gal_mask, bl_dic = bl_dic)

        if (1): #perform fitting
            for dust_sync_iter in range(2):
                if dust_sync_iter == 0:
                    cl_dust_sync = np.copy(cl_dust)
                elif dust_sync_iter == 1:
                    cl_dust_sync = np.copy(cl_sync)
                if dust_sync_iter == 0: 
                    cl_dust_fit = perform_fit(el, cl_dust_sync, ell_norm = ell_norm)
                elif dust_sync_iter == 1: 
                    cl_sync_fit = perform_fit(el, cl_dust_sync, ell_norm = ell_norm)

        print('\n', (freq1, freq2))
        if freq1 == 93:
            delta_ell = 50
        elif freq1 == 145:
            delta_ell = 100
        delta_ell = 50
        el_to_plot = np.arange(min(el), max(el)+1, delta_ell)

        plot_done.append((freq1, freq2))
        #lab = '%s: (%s,%s)' %(which_gal_mask, freq1, freq2)
        lab = r'(%s$\times$%s)' %(freq1, freq2)

        lwval = 1. #lw_dic[freq1]
        colorval = color_dic[(freq1, freq2)]
        alphaval = 1. #alpha_dic[freq1]
        lsval = '-' #ls_dic[freq1]
        #lwval = 1.
        #lab = None
        if cl_dust is not None:
            if show_sync:
                ax = subplot(1,2,1, yscale = 'log', xscale=xscale_val)
            else:
                ax = subplot(111, yscale = 'log', xscale=xscale_val)
            if (1):
                cl_dust_to_plot = np.interp(el_to_plot, el, cl_dust)
                #cl_dust_with_beam_to_plot = np.interp(el_to_plot, el, cl_dust_with_beam)
            else:
                el_to_plot = np.copy(el)
                cl_dust_to_plot = np.copy( cl_dust )
                #cl_dust_with_beam_to_plot = np.copy( cl_dust_with_beam)
            plot(el_to_plot, cl_dust_to_plot, label = lab, lw = 1., linestyle = lsval, color = colorval, alpha = alphaval)
            plot(el, cl_dust_fit, lw = 1., color = colorval, ls = '--')

            ylabel(r'$C_{\ell}\ [\mu {\rm K}^{2}]$', fontsize = 14)
            title(r'Dust: %s' %(which_spec))
            xlim(xmin, xmax); ylim(ymin, ymax)
            xlabel(r'Multipole $\ell$', fontsize = 14)

        if cl_sync is not None and show_sync:
            cl_sync_to_plot = np.interp(el_to_plot, el, cl_sync)                            
            ax = subplot(1,2,2, yscale = 'log', xscale=xscale_val)
            plot(el, cl_sync, label = lab, lw = 0.5, linestyle = lsval, color = colorval, alpha = alphaval)
            #plot(el, cl_sync_fit, lw = 1., color = colorval, ls = '--')
            #plot(el_to_plot, cl_sync_to_plot, label = lab, lw = lwval, linestyle = lsval, color = colorval, alpha = alphaval)
            setp(ax.get_yticklabels(which = 'both'), visible=False)
            title(r'Synchrotron: %s' %(which_spec))

            xlim(xmin, xmax); ylim(ymin, ymax)
            xlabel(r'Multipole $\ell$', fontsize = 14)

if show_sync:
    totcntr = 2
else:
    totcntr = 1
for tmpcntr in range(totcntr):
    if show_sync:
        ax = subplot(1,2,tmpcntr+1, yscale = 'log', xscale=xscale_val)
    else:
        ax = subplot(1,1,tmpcntr+1, yscale = 'log', xscale=xscale_val)
        #text(2000., 1e-5, r'-. Power law fit to 278 GHz and scaled to other bands (T)')
    plot(el_camb, cl_TT, 'gray', lw = 2., alpha = 0.3, zorder = -1000.)#, label = r'%s' %(which_spec))
                
legend(loc = 1, ncol = 2, fontsize = 10)

if (0):
    extra_str_tmp = extra_str.split('/')[0].replace('_', '\_')
    if len(freqarr)>1:
        title(r'%s: %s (%s)' %(which_spec, which_comp.capitalize(), extra_str_tmp), fontsize = 14)
    else:
        title(r'%s: %s (%s): %s GHz' %(which_spec, which_comp.capitalize(), extra_str_tmp, freqarr[0]), fontsize = 14)

    #xlim(None, 200); ylim(1e-6, 1e-2)
suptitle('Sim set = %s' %(which_sim_set.replace('_','\_')), fontsize = 12, y = 1.0)
show();sys.exit()
plfolder = 'reports/galactic_sims/dust_sync_spectra/%s/' %(extra_str)
os.system('mkdir -p %s' %(plfolder))
#savefig('%s/dust_sync_%s.pdf' %(plfolder, which_spec))

In [ ]:
#show beams
for freq in freqarr:
    beam, noise = beam_noise_dic['T'][freq]
    subplot(121)
    plot(bl_dic[freq], color = color_dic[(freq, freq)], label = r'%s GHz: %s$^{\prime}$' %(freq, beam))
    subplot(122)
    plot(bl_dic[145]/bl_dic[freq], color = color_dic[(freq, freq)])
    xlabel(r'Multipole $\ell$', fontsize = 14)
    xlim(xmin, xmax); 
subplot(121)
ylabel(r'B$_{\ell}$', fontsize = 14)
ylim(0., 1.)
legend(loc = 3, ncol = 1, fontsize = 10)
subplot(122)
ylim(0., 50.)
show()